# Playing around with some sparql endpoints and python

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON

## Simple query on dbpedia for testing purposes

In [2]:
endpoint = "http://dbpedia.org/sparql"
dbpedia_default_graph = "http://dbpedia.org"
sparql = SPARQLWrapper(endpoint)
sparql.setReturnFormat(JSON)

In [3]:
# add a default graph, though that can also be part of the query string
sparql.addDefaultGraph(dbpedia_default_graph)

# First, the simplest possible query to see if everything's working
queryString = "SELECT * WHERE { ?s ?p ?o. } LIMIT 3"
sparql.setQuery(queryString)
try :
       result = sparql.query() # result is a stream with the results normally in XML, but we set it to JSON
except :
        print("Could not query!")

result = result.convert()

In [4]:
result

{u'head': {u'link': [], u'vars': [u's', u'p', u'o']},
 u'results': {u'bindings': [{u'o': {u'type': u'uri',
     u'value': u'http://www.w3.org/2002/07/owl#FunctionalProperty'},
    u'p': {u'type': u'uri',
     u'value': u'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'},
    u's': {u'type': u'uri',
     u'value': u'http://dbpedia.org/ontology/deathDate'}},
   {u'o': {u'type': u'uri',
     u'value': u'http://www.w3.org/2002/07/owl#FunctionalProperty'},
    u'p': {u'type': u'uri',
     u'value': u'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'},
    u's': {u'type': u'uri',
     u'value': u'http://dbpedia.org/ontology/birthDate'}},
   {u'o': {u'type': u'uri',
     u'value': u'http://www.w3.org/2002/07/owl#FunctionalProperty'},
    u'p': {u'type': u'uri',
     u'value': u'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'},
    u's': {u'type': u'uri',
     u'value': u'http://dbpedia.org/ontology/acceleration'}}],
  u'distinct': False,
  u'ordered': True}}

## Now supposedly this is simpler when using SPARQLWrapper2 Oo

In [5]:
from SPARQLWrapper import SPARQLWrapper2

# SPARQLWrapper2 automatically uses JSON, so no need to set a return format
sparql = SPARQLWrapper2(endpoint)

queryString = "SELECT ?subj ?prop ?obj FROM <%s> WHERE { ?subj ?prop ?obj. } LIMIT 3" % dbpedia_default_graph
sparql.setQuery(queryString)

try :
    result = sparql.query()
    #print result.variables  # this is an array consisting of "subj" and "prop"
    for binding in result.bindings :
        # each binding is a dictionary. Let us just print the results
        print("s: {subj}\tp: {prop}\to: {obj}".format(subj=binding[u"subj"].value, prop=binding[u"prop"].value, obj=binding[u"obj"].value))
        print("s.type == %s, \t p.type == %s, \t o.type == %s" % (binding[u"subj"].type, binding[u"prop"].type, binding[u"obj"].type))
        #print "%s: %s (of type %s)" % ("s",binding[u"subj"].value,binding[u"subj"].type)
        #print "%s: %s (of type %s)" % ("p",binding[u"prop"].value,binding[u"prop"].type)
    
except:
    print("Could not query!")

s: http://dbpedia.org/ontology/deathDate	p: http://www.w3.org/1999/02/22-rdf-syntax-ns#type	o: http://www.w3.org/2002/07/owl#FunctionalProperty
s.type == uri, 	 p.type == uri, 	 o.type == uri
s: http://dbpedia.org/ontology/birthDate	p: http://www.w3.org/1999/02/22-rdf-syntax-ns#type	o: http://www.w3.org/2002/07/owl#FunctionalProperty
s.type == uri, 	 p.type == uri, 	 o.type == uri
s: http://dbpedia.org/ontology/acceleration	p: http://www.w3.org/1999/02/22-rdf-syntax-ns#type	o: http://www.w3.org/2002/07/owl#FunctionalProperty
s.type == uri, 	 p.type == uri, 	 o.type == uri


## OPTIONAL in SPARQL with usage of [] and in

In [6]:
sparql.addDefaultGraph(dbpedia_default_graph)
queryString = "SELECT ?subj ?o ?opt WHERE { ?subj ?o ?x } LIMIT 3"
sparql.setQuery(queryString)

try:
    result = sparql.query()
    print(result.variables)
    # Now if "opt" has a value as well, there had to be an optional match
    if (u"subj", u"o", u"opt") in result:
        bindings = result[u"subj",u"o",u"opt"]
        # bindings is an array of dictionaries with the full bindings
        for b in bindings :
                subj = b[u"subj"].value
                o    = b[u"o"].value
                opt  = b[u"opt"].value
                print("%s \t %s \t %s" % (subj, o, opt))
                # do something nice with subj, o, and opt
        # another way of accessing to values for a single variable:
        # take all the bindings of the "subj"
        subjbind = result.getValues(u"subj") # an array of Value instances
        
except:
    print("Could not query!")


[u'subj', u'o', u'opt']


# MusicBrainz dataset

In [7]:
endpoint = 'http://linkedbrainz.org/sparql'
sparql = SPARQLWrapper2(endpoint)
# Looked like they had no default graph Oo i dont really get that concept
#sparql.addDefaultGraph()

## Find members of Manowar! Metal!1

In [8]:
query = """ 
PREFIX mo: <http://purl.org/ontology/mo/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>

SELECT ?name ?member
WHERE {
  ?member foaf:name ?name ;
          mo:member_of ?band .
  ?band foaf:name "Manowar"
}
"""
#query = "SELECT ?a ?b ?c WHERE { ?a ?b ?c } LIMIT 3"
sparql.setQuery(query)

try:
    result = sparql.query()
    print("Folgende Variablen sind belegt:")
    print(result.variables)
    for binding in result[u"name", u"member"]:
        print("%s ( %s ) is member of Manowar!!1" % (binding[u"name"].value, binding[u"member"].value))
        
except:
    print("Could not query!")

Folgende Variablen sind belegt:
[u'name', u'member']
Karl Logan ( http://musicbrainz.org/artist/82fe4a25-5cc1-4e6e-ac93-0bd8efbb0cc5#_ ) is member of Manowar!!1
Eric Adams ( http://musicbrainz.org/artist/4c96a93c-85ab-4aaa-83f2-bd1ad759e63b#_ ) is member of Manowar!!1
Joey DeMaio ( http://musicbrainz.org/artist/7db07923-17d5-4353-80e8-f3fed7977576#_ ) is member of Manowar!!1


## Find albums of manowar!

In [9]:
query = """
PREFIX mo: <http://purl.org/ontology/mo/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dc: <http://purl.org/dc/elements/1.1/>

SELECT (STR(?title) AS ?stitle) ?album
WHERE {?band foaf:name "Manowar" ;
             foaf:made ?album .
       ?album a mo:SignalGroup ;
              dc:title ?title}
ORDER BY ?title
"""
sparql.setQuery(query)

try:
    result = sparql.query()
    
    titles = result.getValues(u"stitle")
    albums = result.getValues(u"album")
    for (t,a) in zip(titles, albums):
        print("Manowar made: %s ( %s )" % (t.value, a.value))
        
except:
    print("Could not query!")

Manowar made: 1999-12-04: Moscow, Russia ( http://musicbrainz.org/signal-group/180ba167-1c61-3cfb-8b64-66c8287a9e17#_ )
Manowar made: 2002-12-14: War of the World: Ballsporthalle, Frankfurt, Germany ( http://musicbrainz.org/signal-group/d50af78f-0d0e-34e5-a046-e5e80d2716a5#_ )
Manowar made: An American Trilogy / The Fight for Freedom ( http://musicbrainz.org/signal-group/aa76c495-b835-3d36-bc04-9eac020184cf#_ )
Manowar made: Anthology ( http://musicbrainz.org/signal-group/c917754f-e829-39e2-8d84-f42d83a24f05#_ )
Manowar made: Battle Hymns ( http://musicbrainz.org/signal-group/1d3fb509-0f93-3d67-9e2a-0d5de5786c7f#_ )
Manowar made: Battle Hymns / Sign of the Hammer ( http://musicbrainz.org/signal-group/38ac595d-5c2a-37fd-a36e-35a0a7a3ceeb#_ )
Manowar made: Battle Hymns MMXI ( http://musicbrainz.org/signal-group/f8c18a60-ce65-44e0-828f-11eef7e7f3b0#_ )
Manowar made: Best Collection ( http://musicbrainz.org/signal-group/c13e3530-a338-3abb-b283-847f1293cd52#_ )
Manowar made: Blow Up Your Sp

## Duration of Slayer Albums in minutes

In [ ]:
query = """
PREFIX mo: <http://purl.org/ontology/mo/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>


SELECT DISTINCT (STR(?album_title) AS ?title) ?duration (SAMPLE(?release) AS ?example)
WHERE {?album dc:title ?album_title .
       ?release event:factor ?album ;
                mo:record ?record .
{SELECT ?record ((SUM(?track_duration) / 60000) AS ?duration)
 WHERE { ?record mo:track ?track .
         ?band foaf:made ?track ;
               foaf:name "Slayer" .
         ?track mo:duration ?track_duration
} GROUP BY ?record}
} ORDER BY DESC(?duration)
"""

In [31]:
query = """
PREFIX mo: <http://purl.org/ontology/mo/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dc: <http://purl.org/dc/elements/1.1/>
PREFIX event: <http://purl.org/NET/c4dm/event.owl#>

SELECT DISTINCT (STR(?album_title) AS ?title) ?duration (SAMPLE(?release) AS ?example)
WHERE { 
    ?album dc:title ?album_title .
    ?release event:factor ?album ;
                mo:record ?record .
    {SELECT ?record ((SUM(?track_duration) / 60000) AS ?duration)
    WHERE { 
        ?record mo:track ?track .
        ?band foaf:made ?track ;
            foaf:name "Slayer" .
        ?track mo:duration ?track_duration
    } GROUP BY ?record
}
} ORDER BY DESC(?duration)
"""

In [33]:

#query = "SELECT ?album_title WHERE { ?album_title ?duration ?example } LIMIT 1"
sparql.setQuery(query)
try:
    result = sparql.query()

    #for var_binding in result[u"album_title"]:
    #    print("Album %s" % var_binding[u"album_title"].value)
    
    for var_binding in result[u"title", u"duration", u"example"]:
        t, d, e = var_binding[u"title"].value, var_binding[u"duration"].value, var_binding[u"example"].value
        print("Slayers album %s had a duration of %s minutes!" % (t, d))
        
except Exception:
    print("Could not query!")

Slayers album 2007-06-02: Rock am Ring, Nürburgring, Germany had a duration of 78 minutes!
Slayers album 2001-09-22: Darkness of Christ: Böblingen, Germany had a duration of 77 minutes!
Slayers album Soundtrack to the Apocalypse had a duration of 77 minutes!
Slayers album 1992-08-22: Angel of Death: Monsters of Rock: Donington Park, Castle Donington, UK had a duration of 76 minutes!
Slayers album Soundtrack to the Apocalypse had a duration of 72 minutes!
Slayers album The Sickness Within had a duration of 72 minutes!
Slayers album Seasons in the Abyss / Reign in Blood had a duration of 71 minutes!
Slayers album Soundtrack to the Apocalypse had a duration of 65 minutes!
Slayers album 1991-11-22: Dance of the Dead, Paris, France had a duration of 62 minutes!
Slayers album Slaughter in the North had a duration of 61 minutes!
Slayers album The Big 4: Live From Sofia, Bulgaria had a duration of 61 minutes!
Slayers album Captors of Sin had a duration of 60 minutes!
Slayers album 1998-06-25: 